# CS 3120 Machine Learning HW1

Devon DeJohn, Spring 2020

In [1]:
from HW_3 import k_nearest_neighbors
model = k_nearest_neighbors.Model("data/animals")

In [2]:
model.report()


              precision    recall  f1-score   support

         cat       0.43      0.57      0.49       114
         dog       0.31      0.40      0.35        87
       panda       0.84      0.32      0.47        99

    accuracy                           0.44       300
   macro avg       0.53      0.43      0.44       300
weighted avg       0.53      0.44      0.44       300



In [3]:
model.retrain({"p": 2, "n_neighbors": 7, "weights": "distance"})
model.report()


              precision    recall  f1-score   support

         cat       0.46      0.51      0.49       114
         dog       0.29      0.44      0.35        87
       panda       0.84      0.38      0.53        99

    accuracy                           0.45       300
   macro avg       0.53      0.44      0.45       300
weighted avg       0.54      0.45      0.46       300

